<a href="https://colab.research.google.com/github/daniaqallab/MeZO/blob/main/MeZO_%2B_LoRA_Fine_Tuning_on_RoBERTa_Large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets peft
!pip install git+https://github.com/princeton-nlp/MeZO.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

model_name = "FacebookAI/roberta-large"
model = RobertaForSequenceClassification.from_pretrained(model_name)
tokenizer = RobertaTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [3]:
from datasets import load_dataset

dataset = load_dataset("glue", "mrpc")

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, padding="max_length")

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [5]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,  # Rank of the low-rank decomposition
    lora_alpha=32,  # Scaling factor
    target_modules=["query", "value"],  # Target modules for LoRA
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(model, lora_config)

In [7]:
from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    logging_strategy="steps",     # Log training metrics every `logging_steps`
    logging_steps=10,             # Log every 10 steps
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Reduced batch size to avoid OOM
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.631200,0.555265
2,0.547400,0.498532


Epoch,Training Loss,Validation Loss
1,0.631200,0.555265
2,0.547400,0.498532
3,0.489700,0.472013


TrainOutput(global_step=1377, training_loss=0.5536399225264074, metrics={'train_runtime': 2419.4441, 'train_samples_per_second': 4.548, 'train_steps_per_second': 0.569, 'total_flos': 1.0317107775356928e+16, 'train_loss': 0.5536399225264074, 'epoch': 3.0})

In [8]:
lora_results = trainer.evaluate()
print("LoRA Results:", lora_results)

LoRA Results: {'eval_loss': 0.47201278805732727, 'eval_runtime': 34.9307, 'eval_samples_per_second': 11.68, 'eval_steps_per_second': 1.46, 'epoch': 3.0}


In [13]:
outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=labels)
loss = outputs.loss

In [15]:
inputs = {
    "input_ids": batch["input_ids"].to(device),
    "attention_mask": batch["attention_mask"].to(device),
}
labels = batch["label"].to(device)

In [19]:
outputs = model(**inputs, labels=labels)
print("Outputs:", outputs)
loss = outputs.loss
print("Loss:", loss)
from transformers import AdamW

# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

Outputs: SequenceClassifierOutput(loss=tensor(0.8320, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-0.1338, -0.2458],
        [ 0.3529,  0.2770],
        [-2.5044,  2.2579],
        [-0.9643,  0.7889]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
Loss: tensor(0.8320, device='cuda:0', grad_fn=<NllLossBackward0>)


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
import torch
from tqdm import tqdm

# Move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_dataloader):
        # Move batch to GPU
        inputs = {
            "input_ids": batch["input_ids"].to(device),
            "attention_mask": batch["attention_mask"].to(device),
        }
        labels = batch["label"].to(device)

        # Forward pass
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss

        # Debugging: Print loss
        print("Loss:", loss)

        # Backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Clear GPU cache
    torch.cuda.empty_cache()

  0%|          | 0/917 [00:00<?, ?it/s]

Loss: tensor(0.8204, device='cuda:0', grad_fn=<NllLossBackward0>)


  0%|          | 1/917 [00:00<13:51,  1.10it/s]

Loss: tensor(0.4359, device='cuda:0', grad_fn=<NllLossBackward0>)


  0%|          | 2/917 [00:01<11:37,  1.31it/s]

...

100%|█████████▉| 915/917 [13:00<00:01,  1.17it/s]

Loss: tensor(0.4694, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|█████████▉| 916/917 [13:01<00:00,  1.17it/s]

Loss: tensor(0.2318, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|██████████| 917/917 [13:02<00:00,  1.17it/s]


In [26]:
model.eval()
all_preds = []
all_labels = []
for batch in eval_dataloader:
    # Move batch to GPU
    inputs = {
        "input_ids": batch["input_ids"].to(device),
        "attention_mask": batch["attention_mask"].to(device),
    }
    labels = batch["label"].to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=-1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

from sklearn.metrics import accuracy_score

mezo_accuracy = accuracy_score(all_labels, all_preds)
print("MeZO Accuracy:", mezo_accuracy)

MeZO Accuracy: 0.8308823529411765


In [24]:
from sklearn.metrics import accuracy_score

# Compute accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8309


In [27]:
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_dataloader):
        # Training code here
        ...

    # Evaluate after each epoch
    model.eval()
    all_preds = []
    all_labels = []

    for batch in eval_dataloader:
        inputs = {
            "input_ids": batch["input_ids"].to(device),
            "attention_mask": batch["attention_mask"].to(device),
        }
        labels = batch["label"].to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, dim=-1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute accuracy
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Epoch {epoch + 1} - Accuracy: {accuracy:.4f}")

100%|██████████| 917/917 [00:00<00:00, 1466.48it/s]


Epoch 1 - Accuracy: 0.8309


100%|██████████| 917/917 [00:00<00:00, 1490.23it/s]


Epoch 2 - Accuracy: 0.8309


100%|██████████| 917/917 [00:00<00:00, 1493.12it/s]


Epoch 3 - Accuracy: 0.8309
